# Quickstart Guide: Accessing Cube Data

In the Cube web application (https://cube.jax.org) you can search through various elements such as Studies, Assays, and Data Sets. Using the controls along the top of the table you can filter based on properties to narrow your search. From the "Data Sets" page, for example, you can select data to add to your collection. Once a data set is in your collection you can use the `cube_r` package to access and inspect those data.

The commands in this notebook show how to access data sets in your collection. Other elements, such as studies and assays, can be inspected here as well, but they won't have any associated data - only metadata.

In [1]:
# load required libraries
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(DEBUG)

Loading required package: usethis

Loading required package: reshape


Attaching package: ‘rapportools’


The following objects are masked from ‘package:stats’:

    IQR, median, sd, var


The following objects are masked from ‘package:base’:

    max, mean, min, range, sum



Attaching package: ‘data.table’


The following object is masked from ‘package:reshape’:

    melt


Loading required package: tfruns



## Loggin in

After running the cell below to log in to the Cube service, you will see a link appear in the response. YOU MUST CLICK ON THAT LINK TO COMPLETE THE LOGIN! That will associate this notebook session with your user ID and allow you to pull in data from your collection in https://cube.jax.org/ 

In [2]:
# create a CubeAPI object and log in
cube_api = CubeAPI$new()

# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

INFO [2021-01-08 19:39:13] Version: 0.0.2
INFO [2021-01-08 19:39:13] http://10.105.16.22/metadata-service/
DEBUG [2021-01-08 19:39:13] Cached token length: 788
INFO [2021-01-08 19:39:13] verification_uri_complete: https://thejacksonlaboratory.auth0.com/activate?user_code=SVPJ-PSMD
DEBUG [2021-01-08 19:39:13] verification_uri: https://thejacksonlaboratory.auth0.com/activate
DEBUG [2021-01-08 19:39:13] user_code: SVPJ-PSMD
DEBUG [2021-01-08 19:39:13] device_code: sFVYpPTdifTkyr31QOd2EPEy


In [3]:
storage_info = cube_api$get_metadata_collection_storage_info()

INFO [2021-01-08 19:39:15] GET: http://10.105.16.22/metadata-service/metadata_repository/collection/
INFO [2021-01-08 19:39:16] status_code: 200
DEBUG [2021-01-08 19:39:16] response url: http://10.105.16.22/metadata-service/metadata_repository/collection/?page=1&page_size=100
INFO [2021-01-08 19:39:16] number of accession_ids: 2
INFO [2021-01-08 19:39:16] GET: http://10.105.16.22/metadata-service/metadata_repository/element_instance/
INFO [2021-01-08 19:39:17] status_code: 200
DEBUG [2021-01-08 19:39:17] response url: http://10.105.16.22/metadata-service/metadata_repository/element_instance/?page=1&page_size=100&accession_id=JAXDS0001G&accession_id=JAXDS0001H
INFO [2021-01-08 19:39:17] number of row returned: 2


In [16]:
names(cube_api)

[1] ".__enclos_env__"                     
 [2] "auth0_obj"                           
 [3] "url_base"                            
 [4] "print"                               
 [5] "finalize"                            
 [6] "append_to_path"                      
 [7] "get_disclaimer"                      
 [8] "call"                                
 [9] "post_end_point"                      
[10] "post_element_instance_filter"        
[11] "get_end_point"                       
[12] "get_element_instance_json"           
[13] "get_element_instance"                
[14] "get_data_store_files"                
[15] "parse_uri"                           
[16] "parse_storage_uri"                   
[17] "parse_accession_ids"                 
[18] "get_metadata_collection_storage_info"
[19] "get_metadata_collection"             
[20] "get_element"                         
[21] "login"                               
[22] "initialize"                          
[23] "self"

In [18]:
ls(cube_api$auth0_obj)

[1] "access_token"              "clear_access_token"       
 [3] "device_code"               "expires_in"               
 [5] "finalize"                  "get_access_token"         
 [7] "get_access_token_debug"    "get_device_code"          
 [9] "initialize"                "load_access_token"        
[11] "print"                     "private"                  
[13] "save_access_token"         "self"                     
[15] "user_code"                 "verification_uri"         
[17] "verification_uri_complete"

In [22]:
cube_api$auth0_obj$print()

Auth0DeviceAuth: 
  url_base: 
  .token: 
